## Homework

In this homework, we'll deploy the Straight vs Curly Hair Type model we trained in the 
[previous homework](../08-deep-learning/homework.md).

Download the model from here: 

https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle/model_2024_hairstyle.keras



## Question 1

Now convert this model from Keras to TF-Lite format.

What's the size of the **converted** model?

* 27 Mb
* 43 Mb
* **77 Mb**
* 127 Mb

In [7]:
import tensorflow as tf
from tensorflow import keras
import tensorflow.lite as tflite

In [2]:
model = keras.models.load_model('model_2024_hairstyle.keras')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('model_2024_hairstyle.tflite', 'wb') as f_out:
    f_out.write(tflite_model)


INFO:tensorflow:Assets written to: /var/folders/m5/90bb4mps3dv507n82593klgr0000gn/T/tmpezpi6e2b/assets


INFO:tensorflow:Assets written to: /var/folders/m5/90bb4mps3dv507n82593klgr0000gn/T/tmpezpi6e2b/assets


Saved artifact at '/var/folders/m5/90bb4mps3dv507n82593klgr0000gn/T/tmpezpi6e2b'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 200, 200, 3), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  13458110992: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13458111952: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13458110032: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13458112336: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13459720144: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13459720720: TensorSpec(shape=(), dtype=tf.resource, name=None)


W0000 00:00:1735862784.480385 33718271 tf_tfl_flatbuffer_helpers.cc:392] Ignored output_format.
W0000 00:00:1735862784.481020 33718271 tf_tfl_flatbuffer_helpers.cc:395] Ignored drop_control_dependency.
2025-01-02 16:06:24.482524: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /var/folders/m5/90bb4mps3dv507n82593klgr0000gn/T/tmpezpi6e2b
2025-01-02 16:06:24.483264: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-01-02 16:06:24.483271: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /var/folders/m5/90bb4mps3dv507n82593klgr0000gn/T/tmpezpi6e2b
2025-01-02 16:06:24.486730: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2025-01-02 16:06:24.487348: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-01-02 16:06:24.591231: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle a

In [6]:
!ls -lh

total 484656
-rw-r--r--  1 koda  staff   5.0K Jan  2 16:06 hw9.ipynb
-rw-r--r--@ 1 koda  staff   153M Jan  2 16:01 model_2024_hairstyle.keras
-rw-r--r--@ 1 koda  staff    77M Jan  2 16:06 model_2024_hairstyle.tflite



## Question 2

To be able to use this model, we need to know the index of the input and 
the index of the output. 

What's the output index for this model?

* 3
* 7
* **13**
* 24

In [8]:
interpreter = tflite.Interpreter(model_path='model_2024_hairstyle.tflite')
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']
output_index

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


13